In [1]:
import numpy as np
import xarray as xr

from xarray.core.alignment import align
from scipy import optimize

from ensemblecovariancecomputer import EnsembleCovarianceComputer
from observationoperator import PointObserver
from kalmangaincomputer import KalmanGainComputer
from errorcomputer import ErrorComputer

from localizationfunctions import gaspari_cohn_univariate as gaspari_cohn

In [2]:
## Where are we working
proj_dir = '/Users/zstanley/Documents/git_repos/obs_loc_for_scda'
plot_dir = proj_dir + '/plots/optimal_loc'
my_data_dir = proj_dir + '/my_data/20151206.030000'
nb_dir = proj_dir + '/notebooks'

In [3]:
ds = xr.open_dataset(my_data_dir+'/temperature_covariances_averaged_tropical_pacific.nc')
ds = ds.sel(lon=209.5, lat=12.5)

In [4]:
enscov =  EnsembleCovarianceComputer()
enscov(ds)

/Users/zstanley/Documents/git_repos/obs_loc_for_scda/obs_loc_for_scda/ensemblecovariancecomputer.py:148: UserWarning: Matrix is not positive definite. Smallest eigenvalue is -1.3284245274741694e-09
  warnings.warn('Matrix is not positive definite. Smallest eigenvalue is ' + str(min_eval))


In [5]:
enscov

EnsembleCovarianceComputer:

    Ensemble Covariance Matrices:
        status = set

 --- 
    Ensemble Size           : 20
    Number of Trials        : 1000

In [6]:
sst = PointObserver('sst')
sst(enscov)

In [7]:
sst

EnsembleCovarianceComputer:

    Name of Observation     : sst
    Which Fluid is Observed : ocn
    Which Level is Observed : 0
 --- 
    Pre-defined observation types:
        ['ast', 'sst']

In [37]:
ec = ErrorComputer(enscov)
ec(sst)

/Users/zstanley/Documents/git_repos/obs_loc_for_scda/obs_loc_for_scda/errorcomputer.py:49: RuntimeWarning: divide by zero encountered in true_divide
  loc = np.divide(1, gaspari_cohn(dist, (loc_rad/2)))
/Users/zstanley/Documents/git_repos/obs_loc_for_scda/obs_loc_for_scda/errorcomputer.py:59: RuntimeWarning: divide by zero encountered in true_divide
  loc = np.divide(1, loc_atten * gaspari_cohn(dist, (loc_rad/2)))


In [10]:
import pandas as pd

In [73]:
my_dict = ec.__dict__.copy()

In [74]:
[my_dict.pop(x) for x in ['len_atm','len_ocn', 'slice_atm', 'slice_ocn', 'num_trials']]

[127, 75, slice(0, 127, 1), slice(127, 202, 1), 1000]

In [75]:
for key in my_dict.keys():
    print(key)

error_unloc_atm
error_unloc_ocn
locrad_gcr_atm
locrad_gcr_ocn
error_gcr_atm
error_gcr_ocn
locrad_gcra_atm
locrad_gcra_ocn
locatten_gcra_atm
locatten_gcra_ocn
error_gcra_atm
error_gcra_ocn
error_eorl_atm
error_eorl_ocn


In [50]:
test_this = pd.DataFrame(my_dict, index=['lat', 'lon'])

In [60]:
test_this

,error_unloc_atm,error_unloc_ocn,locrad_gcr_atm,locrad_gcr_ocn,error_gcr_atm,error_gcr_ocn,locrad_gcra_atm,locrad_gcra_ocn,locatten_gcra_atm,locatten_gcra_ocn,error_gcra_atm,error_gcra_ocn,error_eorl_atm,error_eorl_ocn
lat,0.866575,0.263192,0.164149,90.137272,0.060437,0.035429,0.255003,85.111717,0.454592,1.19801,0.058858,0.0351,0.05138,0.033615
lon,0.866575,0.263192,0.164149,90.137272,0.060437,0.035429,0.255003,85.111717,0.454592,1.19801,0.058858,0.0351,0.05138,0.033615


In [61]:
test_this.to_xarray()

<xarray.Dataset>
Dimensions:            (index: 2)
Coordinates:
  * index              (index) object 'lat' 'lon'
Data variables: (12/14)
    error_unloc_atm    (index) float64 0.8666 0.8666
    error_unloc_ocn    (index) float64 0.2632 0.2632
    locrad_gcr_atm     (index) float64 0.1641 0.1641
    locrad_gcr_ocn     (index) float64 90.14 90.14
    error_gcr_atm      (index) float64 0.06044 0.06044
    error_gcr_ocn      (index) float64 0.03543 0.03543
    ...                 ...
    locatten_gcra_atm  (index) float64 0.4546 0.4546
    locatten_gcra_ocn  (index) float64 1.198 1.198
    error_gcra_atm     (index) float64 0.05886 0.05886
    error_gcra_ocn     (index) float64 0.0351 0.0351
    error_eorl_atm     (index) float64 0.05138 0.05138
    error_eorl_ocn     (index) float64 0.03362 0.03362

In [65]:
ds_store = xr.Dataset( {'lat': ds['lat'], 'lon': ds['lon']})

In [57]:
ds_store

<xarray.Dataset>
Dimensions:  ()
Coordinates:
    lon      float32 209.5
    lat      float32 12.5
Data variables:
    *empty*

In [68]:
ds_store2 = xr.Dataset(

    #data_vars=dict(

    #    temperature=(["lat", "lon"], None),

    #    precipitation=(["lat", "lon"], None),

    #),

    coords=dict(

        lon=(["x", "y"], ds['lon']),

        lat=(["x", "y"], ds['lat']),

    ),

    attrs=dict(description="Weather related data."),

)

TypeError: Using a DataArray object to construct a variable is ambiguous, please extract the data using the .data property.

In [59]:
ds_store.loc[dict(lat=12.5, lon=209.5)] = test_this.to_xarray()

KeyError: 'no index found for coordinate lat'